# Computational Cognitive Neuroscience

## Assignment 2 

#### Douwe van Erp (s4258126) & Arianne Meijer - van de Griend (s4620135)

In [ ]:
# imports

### Definition of the different models

TODO explanation neural network

In [ ]:
# deep neural network

TODO explanation convolutional network

In [ ]:
# convolutional neural network with a convolutional layer, a max pooling layer and one fully connected layer

TODO explanation extended network

In [ ]:
# Model iwth additional components (e.g. one of dropout, batch normalization, other activation functions, etc.).

### Performance deep neural networks
TODO visualise, interpret the results 

In [ ]:
# Train test, visualise, etc code

TODO report conclusions

### Performance convolutional neural network
TODO visualise (plot loss), interpret the results + report conclusions

In [ ]:
# Train test, visualise, etc code

TODO report conclusions

### Biological plausibility of convolution
TODO Explain in which ways convolution is biologically plausible and biologically implausible.

### Performance extended model
TODO visualise, interpret the results

In [ ]:
# Train test, visualise, etc code

TODO Report if your new architecture outperforms the original convnet architecture.
Provide a plot and a written explanation of your observed (better/worse) results.